In [1]:
# Import necessary modules
from langchain_community.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import LocalFileStore
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI, OpenAI
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from tqdm.auto import tqdm
from langchain import text_splitter
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from typing import List
from langchain.schema import Document
from uuid import uuid4
from dotenv import load_dotenv
import os
load_dotenv()

/home/eyaya/Desktop/Challenges/Week_6/Automatic_Prompt_Generation _RAG/ragenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
OPENAI_API_KEY = os.environ.get('openai_api_key')

In [3]:
dataset_name = "fka/awesome-chatgpt-prompts"
# Specify the dataset name and the column containing the content
page_content_column = "prompt"   # or any other column you're interested in
# Create a loader instance
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)
# Load the data
data = loader.load()
# Display the first 10 entries
data[:10]

/home/eyaya/Desktop/Challenges/Week_6/Automatic_Prompt_Generation _RAG/ragenv/lib/python3.10/site-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


[Document(page_content='"I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd"', metadata={'act': 'Linux Terminal'}),
 Document(page_content='"I want you to act as an English translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in English. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level English words and sentences. Keep the meaning same, but make them more literary. I want you to only reply the correction, the improvements a

In [4]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)
# Display the first 15 documents
docs[15]

Document(page_content='answer in the corrected and improved version of', metadata={'act': 'English Translator and Improver'})

In [5]:
store = LocalFileStore("./cachce/")

# create an embedder
core_embeddings_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model,
    store,
    namespace = core_embeddings_model.model
)

/home/eyaya/Desktop/Challenges/Week_6/Automatic_Prompt_Generation _RAG/ragenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [6]:
%pip install Chromadb

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [8]:

vectorstore = Chroma.from_documents(docs, embedder, persist_directory="./cachce/")
#base_retriever = vectorstore.as_retriever(search_kwargs={"k" : 2})
# instantiate a retriever
retriever = vectorstore.as_retriever()


In [19]:
template = '''
Break down the prompt genetation step by step based on the following prompt pair examples "Linux Terminal","answer": "I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets [like this]",

    "English Translator and Improver", "I want you to act as an English translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in English. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level English words and sentences. Keep the meaning same, but make them more literary. I want you to only reply the correction, the improvements and nothing else, do not write explanations.",

    "`position` Interviewer","I want you to act as an interviewer. I will be the candidate and you will ask me the interview questions for the `position` position. I want you to only reply as the interviewer. Do not write all the conservation at once. I want you to only do the interview with me. Ask me the questions and wait for my answers. Do not write explanations. Ask me the questions one by one like an interviewer does and wait for my answers. ",

    "JavaScript Console","I want you to act as a javascript console. I will type commands and you will reply with what the javascript console should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets [like this]. ",
    
    "Excel Sheet", "I want you to act as a text based excel. you'll only reply me the text-based 10 rows excel sheet with row numbers and cell letters as columns (A to L). First column header should be empty to reference row number. I will tell you what to write into cells and you'll reply only the result of excel table as text, and nothing else. Do not write explanations. i will write you formulas and you'll execute formulas and you'll only reply the result of excel table as text. "

Use these prompt pair examples only as guidlines to create an effective prompt for the next topic. even if the topic is mensioned before. You will create only prompt for it and not act on the previous description. if the topic is mensioned already, do not use the prompt which you were given, change it.


### CONTEXT
{context}
\n
<bot>:
'''

In [20]:
prompt = PromptTemplate.from_template(template).format(
    context = retriever
)


In [21]:
prompt = ChatPromptTemplate.from_template(prompt)

In [22]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [23]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    |prompt
    | llm
    | StrOutputParser()
    )

In [24]:
query = 'Sql query assistant'

In [25]:
response = chain.invoke('Sql query Assistant')

In [26]:
response

'"I want you to act as a Chroma vector store. I will provide you with words or phrases, and you will retrieve the corresponding vectors from the Chroma vector store. I want you to reply with the retrieved vectors in a code block format, and nothing else. Do not write explanations. Do not provide additional information about the vectors. Simply retrieve and reply with the vectors."'

In [ ]:

def text_split(documents: TextLoader):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    return texts


def embeddings(texts: List[Document]):
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    vectordb = Chroma.from_documents(texts, embeddings)
    return vectordb


In [ ]:
loader = TextLoader('../example.txt')
documents = loader.load()
texts = text_split(documents)
vectordb = embeddings(texts)

In [108]:
template = ''' {prefix}
### CONTEXT
{context}
\n
<bot>:
'''

prompt = PromptTemplate.from_template(template)
formatted_prompt = prompt.format(
    context=retriever,
    prefix=response
)

In [110]:
prompt = ChatPromptTemplate.from_template(formatted_prompt)

In [112]:

llm=OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0.7
    )


In [113]:
retriever=vectordb.as_retriever()
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    |prompt
    | llm
    | StrOutputParser()
    )

In [114]:
chain.invoke("Can you tell me the formula for Linear Regression")

"Sure, I will do my best to answer your questions about the history of mathematics. I will only provide information related to this topic, and I will wait for your response before moving on to the next question.\n\nFirst, let's define mathematics. It is the study of numbers, quantities, and shapes, as well as their relationships and operations. It is considered one of the oldest and most fundamental sciences, with its roots dating back to ancient civilizations such as Egypt, Mesopotamia, and Greece.\n\nNow, let's move on to the history of mathematics. The earliest evidence of mathematical activities can be traced back to prehistoric times, with the use of counting and tallying systems. However, it was the ancient Egyptians who first developed a formal system of mathematics around 3000 BC, primarily for measuring land and constructing buildings.\n\nNext, the Mesopotamians, who lived in the area now known as Iraq, made significant contributions to mathematics. They developed a sexagesima